## Silver Layer Transformations

In [93]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

StatementMeta(, 9fd60a72-30e2-438f-b101-bbcdc9204b58, 95, Finished, Available, Finished)

### Customers

In [94]:
customers = spark.read.table("fabricPulse_lakehouse.Bronze.raw_customers").alias("c")
customers.show(5)

StatementMeta(, 9fd60a72-30e2-438f-b101-bbcdc9204b58, 96, Finished, Available, Finished)

+-----------+--------------------+----------+---------+--------------------+---------------+--------------------+
|customer_id|               email|first_name|last_name|             country|customer_status|          created_at|
+-----------+--------------------+----------+---------+--------------------+---------------+--------------------+
|   bdd640fb|  john21@example.net|       Joy|  Gardner|               Nauru|       VERIFIED|2024-11-08 16:50:...|
|   3b8faa18|hoffmanjennifer@e...|       Amy| Robinson|             Namibia|         BANNED|2024-07-26 17:00:...|
|   ce4a2bbd|helenpeterson@exa...|     Jason|Gallagher|Bosnia and Herzeg...|       VERIFIED|2024-03-13 05:52:...|
|   580d7b71|lindsay78@example...|    Teresa|     Gray|          Micronesia|       VERIFIED|2024-08-06 15:45:...|
|   dc98d2c1|barbara10@example...|    Jeremy|  Roberts|             Belgium|       VERIFIED|2025-09-21 06:55:...|
+-----------+--------------------+----------+---------+--------------------+------------

In [95]:
customers.dtypes

StatementMeta(, 9fd60a72-30e2-438f-b101-bbcdc9204b58, 97, Finished, Available, Finished)

[('customer_id', 'string'),
 ('email', 'string'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('country', 'string'),
 ('customer_status', 'string'),
 ('created_at', 'string')]

In [96]:
customers = customers.withColumn("created_at",to_timestamp(col("created_at")))

StatementMeta(, 9fd60a72-30e2-438f-b101-bbcdc9204b58, 98, Finished, Available, Finished)

### Transactions

In [97]:
transactions = spark.read.table("fabricPulse_lakehouse.Bronze.raw_transactions").alias("t")
transactions.show(5)


StatementMeta(, 9fd60a72-30e2-438f-b101-bbcdc9204b58, 99, Finished, Available, Finished)

+--------------+----------+-------+---------+----------------+------------------+--------+-------------+--------------------+---------------------+
|transaction_id|account_id| amount|direction|transaction_type|transaction_status|currency|instrument_id|           timestamp|parent_transaction_id|
+--------------+----------+-------+---------+----------------+------------------+--------+-------------+--------------------+---------------------+
|  39fda214-4a3|ACC-592910|5861.24|  INBOUND|           TRADE|         COMPLETED|     USD|     INST-004|2025-06-10 02:41:...|                 NULL|
|  FEE-39fda214|ACC-592910|  11.72| OUTBOUND|             FEE|         COMPLETED|     USD|INST-CASH-USD|2025-06-10 02:41:...|         39fda214-4a3|
|  f0918e2b-08a|ACC-583499|3439.13|  INBOUND|           TRADE|         COMPLETED|     USD|     INST-004|2025-02-14 20:34:...|                 NULL|
|  FEE-f0918e2b|ACC-583499|   6.88| OUTBOUND|             FEE|         COMPLETED|     USD|INST-CASH-USD|2025-02-

In [98]:
transactions.dtypes

StatementMeta(, 9fd60a72-30e2-438f-b101-bbcdc9204b58, 100, Finished, Available, Finished)

[('transaction_id', 'string'),
 ('account_id', 'string'),
 ('amount', 'string'),
 ('direction', 'string'),
 ('transaction_type', 'string'),
 ('transaction_status', 'string'),
 ('currency', 'string'),
 ('instrument_id', 'string'),
 ('timestamp', 'string'),
 ('parent_transaction_id', 'string')]

In [99]:
transactions = transactions.select(
    col("transaction_id").cast("string").alias("transaction_id"),
    col("account_id").cast("string").alias("account_id"),
    col("amount").cast("float").alias("amount"),
    col("direction"),col("transaction_type"),col("transaction_status"),col("currency"),
    col("instrument_id").cast("string").alias("instrument_id"),to_timestamp(col("timestamp")).alias("timestamp"),
    col("parent_transaction_id").cast("string")
    )

StatementMeta(, 9fd60a72-30e2-438f-b101-bbcdc9204b58, 101, Finished, Available, Finished)

### Accounts

In [100]:
accounts = spark.read.table("fabricPulse_lakehouse.Bronze.raw_accounts").alias("a")
accounts.show(5)

StatementMeta(, 9fd60a72-30e2-438f-b101-bbcdc9204b58, 102, Finished, Available, Finished)

+----------+------------+--------+--------------+--------------------+-----------+
|account_id|account_type|currency|account_status|           opened_at|customer_id|
+----------+------------+--------+--------------+--------------------+-----------+
|ACC-182155|     SAVINGS|     EUR|        ACTIVE|2024-03-30 18:34:...|   4cac895c|
|ACC-422700|     SAVINGS|     GBP|        ACTIVE|2025-05-21 21:57:...|   dc432ed5|
|ACC-948092|    CHECKING|     USD|        ACTIVE|2025-05-30 15:11:...|   eb9e54e0|
|ACC-333578|    CHECKING|     USD|        ACTIVE|2024-06-17 18:41:...|   d5f25073|
|ACC-542404|     SAVINGS|     EUR|        ACTIVE|2024-03-23 03:55:...|   702cdd20|
+----------+------------+--------+--------------+--------------------+-----------+
only showing top 5 rows



In [101]:
accounts.dtypes

StatementMeta(, 9fd60a72-30e2-438f-b101-bbcdc9204b58, 103, Finished, Available, Finished)

[('account_id', 'string'),
 ('account_type', 'string'),
 ('currency', 'string'),
 ('account_status', 'string'),
 ('opened_at', 'string'),
 ('customer_id', 'string')]

In [102]:
accounts = accounts \
.withColumn("opened_at",to_timestamp(col("opened_at")))

StatementMeta(, 9fd60a72-30e2-438f-b101-bbcdc9204b58, 104, Finished, Available, Finished)

In [103]:
a_valid = accounts.join(customers.select("c.customer_id"),on="customer_id",how="left").withColumn("valid_customer",when(col("a.customer_id").isNull(),False).otherwise(True))

StatementMeta(, 9fd60a72-30e2-438f-b101-bbcdc9204b58, 105, Finished, Available, Finished)

In [104]:
t_valid = transactions \
.join(accounts.select("a.account_id"),on="account_id",how="left") \
.withColumn("valid_account",when(col("account_id").isNull(),False).otherwise(True))

StatementMeta(, 9fd60a72-30e2-438f-b101-bbcdc9204b58, 106, Finished, Available, Finished)

In [105]:
t_valid.show(5)

StatementMeta(, 9fd60a72-30e2-438f-b101-bbcdc9204b58, 107, Finished, Available, Finished)

+----------+--------------+-------+---------+----------------+------------------+--------+-------------+--------------------+---------------------+-------------+
|account_id|transaction_id| amount|direction|transaction_type|transaction_status|currency|instrument_id|           timestamp|parent_transaction_id|valid_account|
+----------+--------------+-------+---------+----------------+------------------+--------+-------------+--------------------+---------------------+-------------+
|ACC-592910|  39fda214-4a3|5861.24|  INBOUND|           TRADE|         COMPLETED|     USD|     INST-004|2025-06-10 02:41:...|                 NULL|         true|
|ACC-592910|  FEE-39fda214|  11.72| OUTBOUND|             FEE|         COMPLETED|     USD|INST-CASH-USD|2025-06-10 02:41:...|         39fda214-4a3|         true|
|ACC-583499|  f0918e2b-08a|3439.13|  INBOUND|           TRADE|         COMPLETED|     USD|     INST-004|2025-02-14 20:34:...|                 NULL|         true|
|ACC-583499|  FEE-f0918e2b| 

### Instruments

In [106]:
instruments = spark.read.table("fabricPulse_lakehouse.Bronze.raw_instruments").alias("i")
instruments.show(5)

StatementMeta(, 9fd60a72-30e2-438f-b101-bbcdc9204b58, 108, Finished, Available, Finished)

+-------------+-----------+--------+--------+--------+-------+
|instrument_id|       name|category|provider|currency| status|
+-------------+-----------+--------+--------+--------+-------+
|     INST-001| Apple Inc.|  EQUITY|  NASDAQ|     USD|TRADING|
|     INST-002| Tesla Inc.|  EQUITY|  NASDAQ|     USD|TRADING|
|     INST-003|    Bitcoin|  CRYPTO| BINANCE|     BTC|TRADING|
|     INST-004|   Ethereum|  CRYPTO| BINANCE|     ETH|TRADING|
|     INST-005|S&P 500 ETF|   INDEX|VANGUARD|     USD|TRADING|
+-------------+-----------+--------+--------+--------+-------+
only showing top 5 rows



In [107]:
instruments.dtypes

StatementMeta(, 9fd60a72-30e2-438f-b101-bbcdc9204b58, 109, Finished, Available, Finished)

[('instrument_id', 'string'),
 ('name', 'string'),
 ('category', 'string'),
 ('provider', 'string'),
 ('currency', 'string'),
 ('status', 'string')]

## Writing to Silver Schema

In [108]:
t_valid.write.format("delta").mode("overwrite").saveAsTable("fabricPulse_lakehouse.silver.transactions")
a_valid.write.format("delta").mode("overwrite").saveAsTable("fabricPulse_lakehouse.silver.accounts")
customers.write.format("delta").mode("overwrite").saveAsTable("fabricPulse_lakehouse.silver.customers")
instruments.write.format("delta").mode("overwrite").saveAsTable("fabricPulse_lakehouse.silver.instruments")

StatementMeta(, 9fd60a72-30e2-438f-b101-bbcdc9204b58, 110, Finished, Available, Finished)